In [1]:
# import packages needed
import numpy as np
import pandas as pd
import re
import nltk
import warnings
warnings.filterwarnings('ignore')
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
nlp = spacy.load('en_core_web_lg')

##### select target news for information extraction

In [10]:
news_for_classification=pd.read_csv('news_for_classification.csv')
# get target news list
target_news_list=list(news_for_classification[news_for_classification['label']==1].news)

##### prepare data for IE

In [12]:
l=[]
# read into data
for news_name in target_news_list:
    temp=[]
    fdir='news/'+news_name+'.txt'# set file name
    with open(fdir,'r') as file:
        news = file.read()
        news_sents = nltk.sent_tokenize(news)
        temp.append(news_name)
        temp.append(news_sents[0])#title
        date=re.findall(r'\d+[a-z]+\d+',news_sents[1])[0]
        temp.append(date)#date
        temp.append(' '.join(news_sents[2:]))#contend
        notes_index=min([i if news_sents[i].find('note')==0 else 6 for i in range(len(news_sents))])
        part_of_contend=' '.join(news_sents[2:notes_index])
        temp.append(part_of_contend)#part of contend
        l.append(temp)

news_for_IE=pd.DataFrame(l,columns=['news','title','date','contend','part_of_contend'])
news_for_IE.head()

news                                              title       date  \
0  news1  germany's mothballed wilhelmshaven to restart ...   2jan2019   
1  news2  italy isab successfully ends 53-day maintenanc...   2jan2019   
2  news3  italian falconara's planned jan 1 works delaye...  02jan2019   
3  news6  italy's sarroch set to start work on cru, rela...   2jan2019   
4  news7  venezuela's pdvsa said to shut distiller no.5 ...  02jan2019   

                                             contend  \
0  refinery: wilhelmshaven, germany . owner: hest...   
1  refinery: isab, italy . owner: lukoil . capaci...   
2  refinery: falconara marittima, italy . owner: ...   
3  refinery: sarroch, sardinia, italy . owner: sa...   
4  refinery: amuay, venezuela . owner: pdvsa . ov...   

                                     part_of_contend  
0  refinery: wilhelmshaven, germany . owner: hest...  
1  refinery: isab, italy . owner: lukoil . capaci...  
2  refinery: falconara marittima, italy . owner: ...  
3  refinery: sarroch, sardinia, italy . owner: sa...  
4  refinery: amuay, venezuela . owner: pdvsa . ov...

In [13]:
#generate news_for_IE needs sometime, can save to local 
#news_for_IE.to_csv('news_for_IE.csv',index=False)

In [14]:
# define a function to to do lemmatization
def get_string_after_lemmatize(x):
    doc=nlp(x)
    new_title=' '.join([word.lemma_ for word in doc]) 
    return(new_title)

In [15]:
# lemmatization of contend and part of contend
news_for_IE['contend_after_lemma']=news_for_IE['contend'].apply(lambda x: get_string_after_lemmatize(x))
news_for_IE['part_of_contend_after_lemma']=news_for_IE['part_of_contend'].apply(lambda x: get_string_after_lemmatize(x))

##### start extract information

In [29]:
# refinery part
def get_refinery_part(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('refinery :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('refinery : '):-2])
    
# overall capacity part
def get_overall_capacity_part1(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('overall capacity :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('overall capacity : '):-2])
    
def get_overall_capacity_part2(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('capacity :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('capacity : '):-2])
    
def get_overall_capacity_part3(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('overall capacity (b/d) :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('overall capacity (b/d) : '):-2])  
    
def get_overall_capacity_part4(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('capacity (b/d) :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('capacity (b/d) : '):-2])

# owner part
def get_owner_part(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('owner :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('owner : '):-2])
    
# unit affected part
def get_unit_affected_part1(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('unit affect :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('unit affect : '):-2])
    
def get_unit_affected_part2(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('unit to be affect :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('unit to be affect : '):-2])
    
def get_unit_affected_part3(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('unit :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('unit : '):-2])
    
# capacity affected part     
def get_capacity_affected_part1(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('unit capacity :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('unit capacity : '):-2])
    
def get_capacity_affected_part2(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('affect capacity :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('affect capacity : '):-2])
    
def get_capacity_affected_part3(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('capacity affect :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('capacity affect : '):-2])
    
# duration part
def get_duration_part(x):
    sents_token=nltk.sent_tokenize(x)
    l=[i if sents_token[i].find('duration :')==0 else 999 for i in range(len(sents_token))]
    if len(set(l))==1:
        return(np.nan)
    else:
        return(sents_token[min(l)][len('duration : '):-2])

In [36]:
news_for_IE['REFINERY']=news_for_IE['contend_after_lemma'].apply(lambda x : get_refinery_part(x))

news_for_IE['OWNER']=news_for_IE['contend_after_lemma'].apply(lambda x : get_owner_part(x))

news_for_IE['OVERALL CAPACITY']=news_for_IE['contend_after_lemma'].apply(lambda x : get_overall_capacity_part1(x))
news_for_IE['OVERALL CAPACITY'].fillna(news_for_IE['contend_after_lemma'].apply(lambda x : get_overall_capacity_part2(x)),inplace=True)
news_for_IE['OVERALL CAPACITY'].fillna(news_for_IE['contend_after_lemma'].apply(lambda x : get_overall_capacity_part3(x)),inplace=True)
news_for_IE['OVERALL CAPACITY'].fillna(news_for_IE['contend_after_lemma'].apply(lambda x : get_overall_capacity_part4(x)),inplace=True)
                                       
news_for_IE['UNIT AFFECTED']=news_for_IE['contend_after_lemma'].apply(lambda x : get_unit_affected_part1(x))
news_for_IE['UNIT AFFECTED'].fillna(news_for_IE['contend_after_lemma'].apply(lambda x : get_unit_affected_part2(x)),inplace=True)
news_for_IE['UNIT AFFECTED'].fillna(news_for_IE['contend_after_lemma'].apply(lambda x : get_unit_affected_part3(x)),inplace=True)
                                    
news_for_IE['CAPACITY AFFECTED']=news_for_IE['contend_after_lemma'].apply(lambda x : get_capacity_affected_part1(x))
news_for_IE['CAPACITY AFFECTED'].fillna(news_for_IE['contend_after_lemma'].apply(lambda x : get_capacity_affected_part2(x)),inplace=True)
news_for_IE['CAPACITY AFFECTED'].fillna(news_for_IE['contend_after_lemma'].apply(lambda x : get_capacity_affected_part3(x)),inplace=True)
                                    
news_for_IE['DURATION']=news_for_IE['contend_after_lemma'].apply(lambda x : get_duration_part(x))

##### result of IE

In [40]:
news_for_IE.iloc[:,-6:].head()

REFINERY                            UNIT AFFECTED  \
0      wilhelmshaven , germany                                      NaN   
1                 isab , italy                                      NaN   
2  falconara marittima , italy  thermal crack unit , vps tower internal   
3   sarroch , sardinia , italy                                      NaN   
4            amuay , venezuela                         distiller unit 5   

                              OWNER  \
0                     hestya energy   
1                            lukoil   
2  api ( anonima petroli italiana )   
3                             saras   
4                             pdvsa   

                                    OVERALL CAPACITY CAPACITY AFFECTED  \
0                   260,000 b / d , close since 2009               NaN   
1  321,000 b / d for isab south and north plant c...               NaN   
2                                       85,000 b / d               NaN   
3             300,000 b / d ( 15 million mt / year )               NaN   
4                                      185,000 b / d               NaN   

                      DURATION  
0                          NaN  
1                          NaN  
2  thirty day start in january  
3                          NaN  
4              since january 2

### Training the NER model

##### prepare for the training data

##### https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/

In [106]:
train_data=[('italy isab successfully ends 53-day maintenance: source .',{'entities':[(36,47,"reason")]}),
           ("italian falconara's planned jan 1 works delayed: source .",{'entities':[(34,39,'reason')]}),
           ("venezuela's pdvsa said to shut distiller no.5 at amuay refinery due to a fire .",{'entities':[(73,77,'reason')]}),
           ("italian falconara's delayed maintenance to start sat: source .",{'entities':[(28,39,'reason')]}),
           ("augusta plant maintenance to start mid-feb for two months: sources .",{'entities':[(14,25,'reason')]}),
           ("thailand's thaioil plans partial refinery turnaround jun-jul .",{"entities":[(42,52,'reason')]}),
           ("spain's bilbao halts sulfur recovery unit for maintenance .",{"entities":[(46,57,'reason')]}),
           ("sweden's lysekil operating normally after glitch resolved: sources .",{"entities":[(42,48,'reason')]}),
           ("hydrocracker at bpcl mumbai restarts after a fire .",{"entities":[(45,49,'reason')]})]


In [114]:
# Adding labels to the `ner`
# Load pre-existing spacy model

nlp=spacy.load('en_core_web_sm')
# Getting the pipeline component
ner=nlp.get_pipe("ner")

for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])
        
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example
# TRAINING THE MODEL
#optimizer = nlp.begin_training()
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(10):

    # shuufling examples  before every iteration
    random.shuffle(train_data)
    losses = {}
    # batch up the examples using spaCy's minibatch
    for batch in spacy.util.minibatch(train_data, size=10):
        for text, annotations in batch:
            # create Example
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            # Update the model
            nlp.update([example], losses=losses, drop=0.3)
            print("Losses", losses)

Losses {'ner': 2.0059264758492525}
Losses {'ner': 4.381345853207431}
Losses {'ner': 8.211082714446954}
Losses {'ner': 10.310626144508978}
Losses {'ner': 12.306064187086365}
Losses {'ner': 17.123648967974543}
Losses {'ner': 19.834249042987437}
Losses {'ner': 21.83262868810202}
Losses {'ner': 23.84455662260875}
Losses {'ner': 1.7920165063605964}
Losses {'ner': 5.485757845576245}
Losses {'ner': 7.272584128094081}
Losses {'ner': 9.220996552117526}
Losses {'ner': 12.913225567890663}
Losses {'ner': 14.910154847197305}
Losses {'ner': 17.50111383894739}
Losses {'ner': 19.464590081446772}
Losses {'ner': 22.02046648293275}
Losses {'ner': 1.4995647029863903}
Losses {'ner': 4.806659918646287}
Losses {'ner': 6.06973689022237}
Losses {'ner': 7.148419760113424}
Losses {'ner': 8.84976435103857}
Losses {'ner': 10.826324387282906}
Losses {'ner': 12.07777931186541}
Losses {'ner': 13.999192295192582}
Losses {'ner': 15.669743956158012}
Losses {'ner': 1.4659284689987544}
Losses {'ner': 2.8290729210420977}
L

In [116]:
doc = nlp(news_for_IE["title"][17])
spacy.displacy.render(doc, style='ent',jupyter=True)